In [ ]:
import os
from dotenv import load_dotenv
import warnings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import StateGraph, END, START
from typing import List
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import pprint


In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
#llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
urls = [
"https://lilianweng.github.io/posts/2023-06-23-agent/",
"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
]

In [ ]:
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

doc_splite = text_splitter.split_documents(docs_list)

In [ ]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splite,
    collection_name="rag-chroma",
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retrieve_blog_posts",
    description="Search and return information about Lilian Weng blog posts on LLM agents , prompt engineering, and adversarial attacks on LLMs."
)

tools = [retriever_tool]

In [ ]:
# Data Model
class GradeDocuments(BaseModel):
    """ Binary score for relevance check on retrieved documents. """
    binary_score : str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeDocuments)

In [ ]:
# Prompt
system = """ You are a grader checking if a document is relevant to a user's question. The check has to be done very strictly..
If the document has words or meanings related to the question, mark it as relevant.
Give a simple 'yes' or 'no' answer to show if the document is relevant or not.
"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

In [ ]:
retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)

In [ ]:
docs

In [ ]:
doc_txt = docs[1].page_content

In [ ]:
doc_txt

In [ ]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [ ]:
question = "Who is Hamid?"

In [ ]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt.pretty_print()

In [ ]:
docs

In [ ]:
rag_chain = prompt | llm

In [ ]:
question = "What is ai agent?"

In [ ]:
# Run
generation = rag_chain.invoke({"context": docs, "question": question})

In [ ]:
generation

### Hallucination Grader

In [ ]:
class GradeHallucinations(BaseModel):
    """ Binary score for hallucination present in generation answer. """

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

In [ ]:
# LLM with function call
structured_llm_grader_1 = llm.with_structured_output(GradeHallucinations)

In [ ]:
# prompt 
system_1 = """
You are a grader checking if an LLM generation is grounded in or sopported by a set of retrieved facts.
Give a simple 'yes' or 'no' answer. 'Yes' means the generation is grounded in or supported by a set of retrieved the facts.
"""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_1),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}")
    ]
)

In [ ]:
hallucination_grader = hallucination_prompt | structured_llm_grader_1

In [ ]:
print(hallucination_grader.invoke({"documents": docs, "generation": generation}))

In [ ]:
# Answer Grader
# data model
class GraderAnswer(BaseModel):
    """ Binary score to assess answer addresses question. """

    binary_score: str = Field(
        description="Answer addresses the question 'yes' or 'no'"
    )

# LLM with function call
structured_llm_grader_2 = llm.with_structured_output(GraderAnswer)

# prompt 
system_2 = """
You are a grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no' . Yes' means that the answer resolves the question.
"""

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_2)
        ("human", "User question \n\n {question} \n\n LLM generation: {generation}")
    ]
)

answer_grader = answer_prompt | structured_llm_grader_2

In [ ]:
print(answer_grader.invoke({"question": question, "generation": generation}))

In [ ]:
system = """You are a question re-writer that converts an input question into a better optimized version for vector store retrieval document.
    You are given both a question and a document.
    - First, check if the question is relevant to the document by identifying a connection or relevance between them.
    - If there is a little relevancy, rewrite the question based on the semantic intent of the question and the context of the document.
    - If no relevance is found, simply return "question not relevant."
    Your goal is to ensure the rewritten question aligns well with the document for better retrieval."""

re_write_prompt = ChatPromptTemplate.from_messages(

        [("system", system),

        (
            "human", """Here is the initial question: \n\n (question} \n,
            Here is the document: \n\n {documents} \n ,
            Formulate an improved question. if possible other return 'question not relevant'."""
        )]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()

In [ ]:
question = "Who is the current president of USA?"

In [ ]:
question_rewriter.invoke({"question": question, "documents": docs})

In [ ]:
class AgentState():
    question : str
    generation : str
    documents : List[str]
    filter_documents: List[str]
    unfilter_documents: List[str]

In [ ]:
def retrieve(state:AgentState):
    print("---RETRIEVE---")
    question = state["question"]
    documents = retriever.get_relevant_documents(question)

    return {"documents": documents, "question": question}

In [ ]:
def grade_documents(state: AgentState):
    print("--- CHECK DOCUMENTS RELEVANCE TO THE QUESTION ---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    unfiltered_docs = []

    for doc in documents:
        score = retrieval_grader.invoke({"question": question, "document": doc})
        grade = score.binary_score

        if grade == "yes":
            print("--- GRADE: DOCUMENT RELEVANT ---")
            filtered_docs.append(doc)

        else:
            print("--- GRADE: DOCUMENT NOT RELEVANT ---")
            unfiltered_docs.append(doc)

    if len(unfiltered_docs) > 1:
        return {"unfilter_documents": unfiltered_docs, "filter_documents":[], "question": question}
    
    else:
        return {"filter_documents": filtered_docs, "unfilter_documents": [], "question": question}

In [ ]:
def decide_to_generate(state:AgentState):
    print("--- ACCESS GRADED DOCUMENTS ---")
    ulfiltered_documents = state["unfilter_documents"]
    filtered_documents = state["filter_documents"]

    if ulfiltered_documents:
        print("--- ALL THE DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY ---")
        return "transform_query"
    
    if filtered_documents:
        print("--- DECISION: GENERATE ---")
        return "generate"

In [ ]:
def generate(state:AgentState):
    print("--- GENERATE ---")
    question = state["question"]
    documents = state["documents"]

    generation = rag_chain.invoke({"content": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [ ]:
def transform_query(state:AgentState):
    question = state["question"]
    documents = state["documents"]

    print(f"this is my document: {documents}")
    response = question_rewriter.invoke({"question": question, "documents": documents})
    print(f"--- RESPONSE --- {response}")

    if response == "question not relevant":
        print("--- QUESTION IS NOT AT ALL RELEVANT ---")
        return {"documents": documents, "question": response, "generation": "question was not at all relevant"}
    
    else:
        return {"documents": documents, "question": response}

In [ ]:
def decide_to_generate_after_transformation(state:AgentState):
    question = state["question"]

    if question == "question not relevant":
        return "query_not_at_all_relevant"
    
    else:
        return "Retriever"

In [ ]:
def grade_generation_vs_documents_and_question(state:AgentState):
    print("--- CHECK HELLUCINATIONS ---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})

    grade = score.binary_score

    # Check hallucinations
    if grade == "yes":
        print("--- DECISION: GENERATION IS GROUNDED IN DOCUMENTS ---")

        print("--- GRADE GENERATION VS QUESTION ---")

        score = answer_grader.invoke({"question": question, "generation": generation})

        grade = score.binary_score

        if grade == "no":
            print("--- DECISION: GENERATION ADDRESS THE QUESTION ---")
            return "useful"
        
        else:
            print("--- DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---TRANSFORM QUERY")
            return "not useful"
        
    else:
        pprint("--- DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---TRANSFORM QUERY")

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("Docs_Vector_Retrieve", retrieve)
workflow.add_node("Grading_Generated_Documents", grade_documents)
workflow.add_node("Content_Generator", generate)
workflow.add_node("Transform_User_Query", transform_query)

In [ ]:
workflow.add_edge(START, "Docs_Vector_Retrieve")
workflow.add_edge("Docs_Vector_Retrieve", "Grading_Generated_Documents")
workflow.add_conditional_edges("Docs_Vector_Retrieve",
                               decide_to_generate,
                                {
                                    "generate": "Content_Generator",
                                    "transform_query": "Transform_Query"
                                })

workflow.add_conditional_edges(
    "Content_Generator",
    grade_generation_vs_documents_and_question,
    {
        "useful": END,
        "not useful": "Transform_User_Query"
    }
)

workflow.add_conditional_edges(
    "Transform_User_Query",
    decide_to_generate_after_transformation,
    {
        "Retriever": "Docs_Vectore_Retrieve",
        "query_not_at_all_relevant": END
    }
)

In [ ]:
app = workflow.compile()

In [ ]:
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
inputs = {"question": "Explain how the different types of agent memory work"}

In [ ]:
app.invoke(input=inputs)["generation"]

In [ ]:
inputs = {"question": "Who is a prompt engineering?"}

In [ ]:
app.invoke(input=inputs)["generation"]

In [ ]:
inputs = {"question": "Who is the first president of USA?"}

In [ ]:
app.invoke(input=inputs)["generation"]